# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
db_list = mongo.list_database_names()
pprint(db_list)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
collection_names = db.list_collection_names()
print("Collections in 'uk_food' database:")
for name in collection_names:
    print(name)

In [ ]:
# review the collections in our new database
collections = db.list_collection_names()
print("Collections in the 'uk_food' database:")
for collection in collections:
    print(collection)

In [ ]:
# review a document in the establishments collection
establishment_document = db.establishments.find_one()
print("Sample document from the 'establishments' collection:")
pprint(establishment_document)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": 1,  #same as other restaurants
    "AddressLine1": "123 Example Street",  # Replaced address
    "AddressLine2": "Example Area",
    "AddressLine3": "Greenwich",
    "PostCode": "EX4 MPL",  # Replaced postcode
    "RatingValue": None, 
    "RatingKey": "Not Yet Rated",
    "LocalAuthorityName": "Greenwich", }

In [ ]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant_data)

In [ ]:
# Check that the new restaurant was inserted
print("New restaurant added with ID:", insert_result.inserted_id)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

for doc in establishments.find(query, projection):
    print(doc)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
filter_criteria = {"BusinessName": "Penang Flavours"}

update_data = {"$set": {"BusinessTypeID": 1}}

update_result = establishments.update_one(filter_criteria, update_data)


In [ ]:
# Confirm that the new restaurant was updated
print(f"Documents matched: {update_result.matched_count}")
print(f"Documents updated: {update_result.modified_count}")

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName as 'Dover': {dover_count}")

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f"Documents deleted: {delete_result.deleted_count}")

In [ ]:
# Check if any remaining documents include Dover
remaining_dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Remaining documents with LocalAuthorityName as 'Dover': {remaining_dover_count}")

In [ ]:
# Check that other documents remain with 'find_one'
sample_document = establishments.find_one()
print("Sample document from remaining collections:")
print(sample_document)

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
update_result = establishments.update_many(
    {},  # An empty filter matches all documents
    [
        {"$set": {
            "geocode.latitude": {"$toDecimal": "$geocode.latitude"},
            "geocode.longitude": {"$toDecimal": "$geocode.longitude"}
        }}
    ]
)

print(f"Documents updated: {update_result.modified_count}")

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
update_result = establishments.update_many(
    {"RatingValue": {"$ne": None}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)
print(f"Documents updated (RatingValue converted to integer): {update_result.modified_count}")

In [ ]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments.find_one({"RatingValue": {"$ne": None}})
print("Sample document for verification:")
print(sample_document)